# TIE Reconstruction Template
In this notebook we demonstrate solving the transport of intensity equation (TIE) and finding magnetic induction from an experimental through focal series (tfs) of TEM images. For more information please check out our [wiki/documentation pages](https://pylorentztem.readthedocs.io/en/latest/). 

If you have never used a Jupyter Notebook before we recommend familiarizing yourself with the [notebook basics](https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Notebook%20Basics.html). While Python knowledge is not strictly required for running PyLorentz code, a certain familiarity is helpful for understanding how to modify the examples to suit your needs. 

This notebook is ready to be run as-is. The first time, however, you will need to download the example data. This can be done through the notebook and is demonstrated below. Cells can be executed by selecting and pressing ``shift + enter``, and you can proceed with running all of the cells in order. The python docstrings for these functions can be found on the wiki, but are also viewable in Jupyter. E.g. for the TIE() function, executing a cell containing: `?TIE` will pull up the documentation for that function. 

All of the features available in this notebook are also available in the TIE GUI, which additionally contains processes for aligning experimental datasets. For details on running that please refer to the README in the ``Pylorentz/GUI/`` directory. 
  
Authors: Arthur McCray, CD Phatak  
V1.0, ANL, July 2020

### Importing the necessary modules

In [ ]:
import sys

sys.path.append("../PyTIE")
from microscopes import Microscope
from TIE_helper import *
from TIE_reconstruct import TIE, SITIE

%matplotlib widget

## Downloading the data

This first section shows how to download example data from the [Materials Data Facility](https://doi.org/10.18126/z9tc-i8bf). This experimental dataset contains a full through focus series (tfs) of an artificial spin ice structure with the sample in both flipped and unflipped orientations. 

The files will be placed in `Examples/cphatak_pytie_example_v1.1/`, and you only need to run this following cell the first time through the notebook. Incidentally there seems to be a bug in the mdf-forge package and the progress bar doesn't move, but it generally takes ~5-10 minutes for the download depending on your internet speed. 

If you are having trouble downloading the data please make sure 
* You have a Globus endpoint on your computer
* You have Globus Connect running

If it still isn't working you can download the data with globus from the [data page](https://doi.org/10.18126/z9tc-i8bf). Click "Globus" under "Get the Data" and download using their online app. 

In [ ]:
from mdf_forge import Forge
mdf = Forge()

# Match the dataset by title and only collect "dataset" resource types
res = mdf.match_resource_types('dataset').match_titles('pytie').search()

# Download the data using Globus
mdf.globus_download(res, download_datasets=True)

## Loading the data into PyTIE
Now that we have our data on the computer, we need to load it into PyTIE with the `load_data()` function. We set a few parameters here: 

- path (str): We specify the filepath as the one created with the example data.
- fls_file (str): The name of the .fls file, which itself contains the image filenames and defocus values. 
- flip_fls_file (str): We explicitly show that there isn't a flip fls file, though this could be omitted as it's the default value. 
- al_file (str): The name of the aligned image file. 
- flip (bool): Whether or not there is a flip tfs along with the unflip. 

In [ ]:
data_loc = './cphatak_pytie_example_v1.1/'
fls_name = "dataset1.fls"
flip_fls_name = None # just one fls file
stack_name = 'd1_full_align.tif' 
flip = True # there is/isn't a flipped tfs and flipstack

imstack, flipstack, ptie = load_data(path=data_loc,
                                          fls_file=fls_name, 
                                          flip_fls_file=flip_fls_name,
                                          al_file=stack_name, 
                                          flip=flip)

print("Defocus values: ", ptie.defvals, "nm")
# create microscope object: 
#Only relevant parameter for TIE reconstruction is accelerating voltage. 
pscope = Microscope(E=200e3)

Now view the masked infocus image.


In [ ]:
# Scroll through and view the aligned images 
show_stack(imstack + flipstack)

In [ ]:
show_im(ptie.infocus, title='Masked infocus image', scale=ptie.scale)

## Select a smaller image size (optional)
There may be times when you would want to only reconstruct part of an image, e.g. because a window edge introduces a high-contrast edge that interferes with the reconstruction. Although this is available here in the Jupyter notebook, it is somewhat finnicky. We recommend using the GUI if you want to be precise with region selection, as it also allows for rotating the images. 

The select_region() method of the TIE_params class opens an interface to select a region, and running the crop_ims() method saves it. So you can change the rectangular region after running crop_ims() but the changes will not take effect until it is run again. 
  
* If you choose a non-square subregion, the results are no longer quantitative. Qualitatively they will appear correct except beyond aspect ratios of ~3:1, where Fourier edge effects become more prominent. These can be reduced by symmetrizing the image.  
* Default is 1/4 image size centered in the middle
* Drag mask and corners to get the section you want to reconstruct
    - It's finnicky, if the corner isn't dragging, try moving mouse cursor more slowly and make sure you're on the corner. 
    - if it still isn't working you can change ptie.crop by hand which sets the boundary (in pixels) for which to crop.



In [ ]:
ptie.select_region()

In [ ]:
### Run this when the region is what you want. 
ptie.crop_ims()

In [ ]:
print(f"Ptie.crop: {ptie.crop}")
show_stack(ptie.imstack+ptie.flipstack, ptie)

## The actual phase reconstruction
Here we perform the TIE reconstruction. The following parameters are set: 

* Set which focus values you want to use with index i 
* images will be saved as /dataloc/images/dataname_focusval_imagename.tif
* if you want to symmetrize the images set sym = True
* if you want to run with a Tikhonov frequency
    - qc = XX, allows you to input a frequency value in 1/nm
    - qc = None and it will run without a filter
* Save options:  
    - True    -- saves all images results directory
    - 'b'     -- saves 'color_b', 'bxt', 'byt'
    - 'color' -- saves just 'color_b'
    - False   -- Does not save images, but they can still be viewed in the notebook
    - If any images are saved, a dataname_params.txt file will be saved as well giving reconstruction parameters. 

In [ ]:
print('actual defocus values: ', ptie.defvals)
print('reconstruct with   i = ', '    0    ', '   1   ', '     2 ...')

In [ ]:
i = 2 # Select which defocus value you'll be working with if 3-point deriv 
dataname = 'example_TIE'# str
sym = False # bool 
qc = False # a tikhonov filter isn't necessary for this data
save = 'b' # str or bool 
print(f'Set to reconstruct for defocus value: {ptie.defvals[i]} nm ')

In [ ]:
results = TIE(i, ptie, pscope, 
                     dataname = dataname, 
                     sym=sym, 
                     qc = qc, 
                     save=save)

## Viewing the images
You can of course just look at the data where it's saved, but if you don't want to save it and move it to a local computer you can view the images here, as they're all in the results dictionary. 
* Results: 
    - 'byt' : y-component of integrated magnetic induction,
    - 'bxt' : x-component of integrated magnetic induction,
    - 'bbt' : magnitude of integrated magnetic induction, 
    - 'phase_b' : magnetic phase shift (radians),
    - 'phase_e' : electrostatic phase shift (if using flip stack) (radians),
    - 'dIdZ_m' : intensity derivative for calculating phase_m, (useful for troubleshooting)
    - 'dIdZ_e' : intensity derivative for calculating phase_e (if using flip stack), 
    - 'color_b' : RGB image of magnetic induction,
    - 'inf_im' : the in-focus image

In [ ]:
show_im(results['phase_b'], title='magnetic phase shift', scale=ptie.scale, 
        cbar_title="Radians")

In [ ]:
show_im(results['phase_e'], 'electrostatic phase shift',scale=ptie.scale, 
        cbar_title="Radians")

Show the image with a four-fold colorwheel instead of a 3-fold hsv colormap

In [ ]:
from colorwheel import color_im
show_im(color_im(results['bxt'], results['byt'], hsvwheel=False), "Magnetic induction with four-fold colormap", simple=True)

Show a vector plot of the magnetic induction over the colormap image. The arrows size is fixed so you will have to zoom in to see them. There are few parameters required to have the vectormap look good. 
- a (int): Number of arrows to show in the x and y directions
- l (float): Scale factor of arrows. Larger l -> shorter arrows. 
- w (float): Width scaling of arrows.
- title (str): (*optional*) Title for plot. Default None. 
- color (bool): (*optional*) Whether or not to show a colormap underneath the arrow plot.
- hsv (bool): (*optional*) Only relevant if color == True. Whether to use an hsv or 4-fold colorwheel in the color image. 
- origin (str): (*optional*) Control image orientation. 
- save (str): (*optional*) Path to save the figure. 

In [ ]:
show_2D(results['bxt'], results['byt'], a=100, l=1, w=1, title='Arrow overlay image', color=True, hsv=True,
        origin='upper', save = ptie.data_loc+'images/arrow_colormap.png')

### Single Image Reconstruction (SITIE)
Uniformly thin magnetic samples can be reconstructed from a single defocused image if their only source of contrast is magnetic Fresnel contrast. As this sample is an island structure that is not the case, and we require two TFS to reconstruct the phase. For an example of SITIE on simulated data please refer to SIM_template.ipynb

--- end notebook ---